In [ ]:
import gzip    # to unzip the dataset
import cPickle # to load the dataset
import random  # for random initialization of weights and biases
import numpy as np

# Import the MNIST dataset

In [ ]:
f = gzip.open('mnist.pkl.gz', 'rb')
training_data, validation_data, test_data = cPickle.load(f)
f.close()

Return the MNIST data as a tuple containing the training data, the validation data, and the test data.

The 'training_data' is returned as a tuple with two entries.
The first entry contains the actual training images of hand-written digits, while the second entry contains the digit values corresponding to the training images.
In the 'training_data' there are a total of 50,000 images and their correct classification.
Each image is represented by its 28 * 28 = 784 pixels, while the classification is just an integer value between 0 and 9.

The 'validation_data' and 'test_data' have the same structure, except each contains only 10,000 images.

We use the following function to modify the format of the second entry of 'training_data'.

In [ ]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

Return a 10-dimensional unit vector with a 1 in the jth position and zeroes elsewhere. This is used to convert a digit into a corresponding desired output from the neural network, i.e., 2 --> (0,0,1,0,0,0,0,0,0,0).

We apply 'vectorized_result' to the labels of the training set. This is better for the network. In general, that is common practice in classification problems.

In [ ]:
training_inputs = [np.reshape(x, (784, 1)) for x in training_data[0]]
training_results = [vectorized_result(y) for y in training_data[1]]
training_data = zip(training_inputs, training_results)
validation_inputs = [np.reshape(x, (784, 1)) for x in validation_data[0]]
validation_data = zip(validation_inputs, validation_data[1])
test_inputs = [np.reshape(x, (784, 1)) for x in test_data[0]]
test_data = zip(test_inputs, test_data[1])

Return a tuple containing '(training_data, validation_data, test_data)'.

In particular, 'training_data' is a list containing 50,000 2-tuples (x, y), where x is a 784-dimensional numpy.ndarray containing the input image and y is a 10-dimensional numpy.ndarray with all zeroes and a 1 in the right place.

'validation_data' and 'test_data' are lists containing 50,000 2-tuples (x, y), where x is a 784-dimensional numpy.ndarray containing the input image and y is a digit corresponding to the correct digit for x.

In [ ]:
#show first element in each set
print "training: {0}".format(training_data[0])
print "validation: {0}".format(validation_data[0])
print "test: {0}".format(test_data[0])

SKIP NEXT LINE IF YOU WANT TO USE ALL THE TRAINING DATA (you probably want to skip it).

In [ ]:
#halves the training data
training_data=training_data[:25000][:25000]

# Setup the Network

Define the activation function (sigmoid) and its derivative used in backpropagation.

In [ ]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Setup of a generic feed-forward network and the functions necessary to apply stochastic gradient descent.

In [ ]:
class Network(object):

    def __init__(self, sizes):
        
        """The list 'sizes' contains the number of neurons in the respective layers of the network.
        In our network we will work with an input layer of size 784 and an output layer of size 10.
        
        The biases and weights for the network are initialized randomly, using a Gaussian distribution
        with mean 0, and variance 1."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
        self.accuracy = [] #list to store the accuracy over the epochs
        self.cost_val = [] #list to store the cost over the epochs
        
        """remember old nabla if using momentum"""
        #self.old_nabla_w = [np.zeros(w.shape) for w in self.weights]

    def feedforward(self, a):
        
        """Return the output of the network given the input 'a'."""
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        
        """Train the neural network using mini-batch stochastic gradient descent.
        
        If 'test_data' is provided then the network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for tracking progress, but slows things
        down substantially."""
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            self.cross_entropy() #used to save the cost over the epochs in a vector when using the cross entropy
            #self.quadratic()     #used to save the cost over the epochs in a vector when using the quadratic cost function
            
            if test_data:
                print "Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        
        """Update the network's weights and biases by applying gradient descent using backpropagation to a single mini batch.
        The 'mini_batch' is a list of tuples (x, y), and 'eta' is the learning rate."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        """Use only one of the following options when running the code! Each one corresponds
        to a different choice of regularization"""
        
        """SGD"""
        #self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        
        """SGD + L2 regularization (weight decay) with lambda = 5"""
        self.weights = [(1-eta*5/len(training_data))*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        
        """SGD + L1 regularization with lambda = 5"""
        #self.weights = [w-eta*5/len(training_data)*np.sign(w)-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        
        """SGD + momentum with lambda = 0.1"""
        #self.weights = [w + 0.1*old_nw -(eta/len(mini_batch))*nw
                       #for w, nw, old_nw in zip(self.weights, nabla_w, self.old_nabla_w)]
        #self.old_nabla_w = nabla_w    """remember to enable self.old_nabla_w in the initialization!"""
        
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
        
        

    def backprop(self, x, y):
        
        """Return a tuple '(nabla_b, nabla_w)' representing the gradient of the cost function."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        """feedforward"""
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        """backward pass"""
        delta = (activations[-1] - y) # * sigmoid_prime(zs[-1])  #Enable the derivative of the sigmoid only if
                                                                 #you want to use the quadratic cost function.
                                                                 #This term cancels out when using the cross-entropy.
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        
        """Return the number of test inputs for which the neural network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever neuron in the final layer has the highest activation."""
        
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        evaluation = sum(int(x == y) for (x, y) in test_results)
        
        self.accuracy.append(evaluation)
        
        return evaluation
    
    def quadratic(self):
        """Remembers the cost of the quadratic cost function at each epoch."""
        val_cost = [np.multiply((self.feedforward(x)-vectorized_result(y)),(self.feedforward(x)-vectorized_result(y)))
                   for (x, y) in validation_data]
        Sum_val = sum([sum(x[0]) for x in val_cost])
        
        newCost_val = Sum_val/(2*len(validation_data))
        self.cost_val.append(newCost_val)

    def cross_entropy(self):
        """Remember the value of the cross entropy at each epoch."""
        val_cost = [np.multiply(vectorized_result(y), np.log(self.feedforward(x))).transpose()
                   for (x, y) in validation_data]
        Sum_val = sum([sum(x[0]) for x in val_cost]) 
                     
        newCost_val = -Sum_val/len(validation_data)
        self.cost_val.append(newCost_val)

# Let's do some training

For this project we initialize an instance of the class 'Network' with input layer of size 784 (number of pixels in an image), output layer of size 10 (remember: image classification is a 10-dimensional unit vector). There can be any number of hidden layers of any dimension in between.

In [ ]:
net = Network([784, 100, 10])

In [ ]:
net.SGD(training_data, 30, 10, 0.5, validation_data)

Train the network with stochastic gradient descent, choosing the number of epochs, the size of the minibatches and the learning rate. You can also choose a test set to print the accuracies after each epoch. If no such set is chosen, then the training happens anyway but the accuracy reached after each epoch is not evaluated.

Plot a graph showing the cost over the epochs for the validation set.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x=list(range(0, len(net.cost_val)))
y=net.cost_val

plt.plot(x,y,label = "eta=0.5")

plt.legend()

plt.xlabel('epochs')
plt.ylabel('cost')

plt.grid(True)
plt.show()

Plot a graph showing the evolution of the accuracy over the epochs.

In [ ]:
x=list(range(0, len(net.accuracy)))
y=net.accuracy

plt.plot(x,y)

plt.xlabel('epochs')

plt.ylabel('accuracy')

plt.grid(True)
plt.show()

Accuracy on test set.

In [ ]:
net.evaluate(test_data)

Train more with a smaller learning rate.

In [ ]:
net.SGD(training_data, 30, 10, 0.1, test_data)

Plot a confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pred_labels = [np.argmax(net.feedforward(x)) for (x, y) in test_data]
true_labels = [y for (x, y) in test_data]

conf_matrix = confusion_matrix(true_labels, pred_labels)

df_cm = pd.DataFrame(conf_matrix, range(10), range(10))
plt.figure(figsize=(14,10))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()